#DATAMINING PROJECT V1.8
***From Baptiste Danichert & Ahmed Balibala***

**Last update**: getting a better score

**Description:** You have noticed that to improve one’s skills in a new foreign language, it is important to read texts in that language. These texts have to be at the reader’s language level. However, it is difficult to find texts that are close to someone’s knowledge level (A1 to C2). You have decided to build a model for English speakers that predicts the difficulty of a French written text. This can be then used, e.g., in a recommendation system, to recommend texts, e.g, recent news articles that are appropriate for someone’s language level. If someone is at A1 French level, it is inappropriate to present a text at B2 level, as she won’t be able to understand it. Ideally, a text should have many known words and may have a few words that are unknown so that the person can improve.



# 1. Loading the training data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/BapDanSI/DataMiningProject/main/data/training_data.csv')
df_pred = pd.read_csv('https://raw.githubusercontent.com/BapDanSI/DataMiningProject/main/data/unlabelled_test_data.csv')

In [ ]:
df_pred.head()

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."


In [ ]:
df.head()

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


# 2. Dataframe analysis


In [ ]:
df.shape

(4800, 3)

-> 4800 rows and 2 columns (excluding first column "id")

In [ ]:
df.isnull().sum()

id            0
sentence      0
difficulty    0
dtype: int64

-> no NAs

In [ ]:
df.duplicated(subset="sentence").value_counts()

False    4800
dtype: int64

-> no duplicate in the data

#3.  Baseline

In [ ]:
np.random.seed = 0

In [ ]:
base_rate = max(df.value_counts('difficulty'))/df.shape[0]
print('Base rate:', round(base_rate,4))

Base rate: 0.1694


# 4. Tokinizer function

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install -U wordstats
!python -m spacy download fr_core_news_sm
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stopwords
from wordstats import Word, common_words
from collections import Counter
import string
import nltk
import spacy
nlp = spacy.load("fr_core_news_sm")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.9 MB/s 
     |████████████████████████████████| 1.2 MB 50.3 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for wordstats: filename=wordstats-1.0.7-py3-none-any.whl size=3616363 sha256=4f57d0304b169d632f234e9e385bffc3b1567a04adc253856d2b01f277387307
  Stored in directory: /root/.cache/pip/wheels/07/27/7d/24ade697c516ed02f369a90c5fec463286a6a67a299f75b711
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34527 sha256=182d7b4a46291286a6287a78dcf332e3b0923848d5e877a3074af482b238b176
  Stored in directory: /root/.cache/pip/wheels/e0/ea/e1/7b17c0af6f0228de33d82556dbad4fe823d90696fc50f2c81d
Successfully built wordstats configobj
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWar

In [ ]:
df['n_diff'] = df['difficulty'].replace(['A1', 'A2', 'B1', 'B2', 'C1', 'C2'], [1,2,3,4,5,6])

In [ ]:
punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
stop_words=spacy.lang.fr.stop_words.STOP_WORDS

# Create tokenizer function
def spacy_tokenizer(sentence):
    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]


    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # Return preprocessed list of tokens
    return mytokens

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer) # we use the above defined tokenizer

# 5. Classification Algorithms

Dependent variable (y) is the column named "difficulty".
<br>We split the data into 80% training and 20% test set.


In [ ]:
y = df['difficulty']
X = df["sentence"]
X_pred = df_pred["sentence"]

### i. Logistic Regression.
We use the following parameters for the LogisticRegressionCV():

* cross-validation to 5 folds
* maximum interation to 1000
* random state to 0

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
np.random.seed = 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

classifier = LogisticRegression()

pl = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

pl.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x7f4fca98b9d0>)),
                ('classifier', LogisticRegression())])

In [ ]:
def evaluate(true, pred):
    precision = precision_score(true, pred,average='macro')
    recall = recall_score(true, pred,average='macro')
    f1 = f1_score(true, pred,average='macro')
    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

In [ ]:
y_pred = pl.predict(X_test)

evaluate(y_test, y_pred)

CONFUSION MATRIX:
[[71 45 22  7  8  8]
 [44 54 31 13 12 10]
 [25 29 49 21 14 22]
 [10 11 16 56 25 26]
 [15  4 17 25 72 40]
 [ 7  6 12 23 31 79]]
ACCURACY SCORE:
0.3969
CLASSIFICATION REPORT:
	Precision: 0.3944
	Recall: 0.3969
	F1_Score: 0.3948


In [ ]:
predict = pl.predict(X_pred)

In [ ]:
submission= pd.DataFrame()
submission['id']= df_pred.index
submission['difficulty'] = predict

In [ ]:
submission.to_csv("submission.csv", index=False)

### ii. kNNeighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

grid = {'n_neighbors':np.arange(1,100),
        'p':np.arange(1,3),
        'weights':['uniform','distance']}

knn = KNeighborsClassifier()
classifier_knn = GridSearchCV(knn, grid, cv=5)

pl_knn = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier_knn)])

pl_knn.fit(X_train, y_train)

print("Hyperparameters:", classifier_knn.best_params_)

Hyperparameters: {'n_neighbors': 91, 'p': 2, 'weights': 'distance'}


In [ ]:
y_knn_predict = pl_knn.predict(X_test)
evaluate(y_test, y_knn_predict)

CONFUSION MATRIX:
[[131   2   1   0  22   5]
 [132   6   5   0  14   7]
 [121   5  13   1  10  10]
 [ 91   0   7   2  24  20]
 [110   1   5   2  30  25]
 [104   0   7   2  11  34]]
ACCURACY SCORE:
0.2250
CLASSIFICATION REPORT:
	Precision: 0.3089
	Recall: 0.2223
	F1_Score: 0.1679


In [ ]:
knn_predict = pl_knn.predict(X_pred)
submission_knn= pd.DataFrame()
submission_knn['id']= df_pred.index
submission_knn['difficulty'] = knn_predict
submission_knn.to_csv("submissionknn.csv", index=False)

###  iii. Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
pl_dtc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', dtc)])
pl_dtc.fit(X_train, y_train)
y_pred_dtc = pl_dtc.predict(X_test)

evaluate(y_test, y_pred_dtc)

CONFUSION MATRIX:
[[90 40 25  3  2  1]
 [63 60 29  7  2  3]
 [38 40 47 16  8 11]
 [21 20 29 31 14 29]
 [23 17 29 28 41 35]
 [24 18 17 30 25 44]]
ACCURACY SCORE:
0.3260
CLASSIFICATION REPORT:
	Precision: 0.3325
	Recall: 0.3249
	F1_Score: 0.3174


In [ ]:
dtc_predict = pl_dtc.predict(X_pred)
submission_dtc= pd.DataFrame()
submission_dtc['id']= df_pred.index
submission_dtc['difficulty'] = dtc_predict
submission_dtc.to_csv("submissiondtc.csv", index=False)

### iv. Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
pl_rfc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', rfc)])

pl_rfc.fit(X_train, y_train)

y_pred_rfc = pl_rfc.predict(X_test)

evaluate(y_test, y_pred_rfc)

CONFUSION MATRIX:
[[111  31  14   3   1   1]
 [ 92  49  16   4   2   1]
 [ 50  40  44  13   5   8]
 [ 40  14  21  40   9  20]
 [ 44  13  17  29  49  21]
 [ 34   9  11  22  18  64]]
ACCURACY SCORE:
0.3719
CLASSIFICATION REPORT:
	Precision: 0.4119
	Recall: 0.3715
	F1_Score: 0.3664


In [ ]:
rfc_predict = pl_rfc.predict(X_pred)
submission_rfc= pd.DataFrame()
submission_rfc['id']= df_pred.index
submission_rfc['difficulty'] = rfc_predict
submission_rfc.to_csv("submissionrfc.csv", index=False)

### v. Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

svc = SVC()
pl_svc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', svc)])

pl_svc.fit(X_train, y_train)

y_pred_svc = pl_svc.predict(X_test)

evaluate(y_test, y_pred_svc)

CONFUSION MATRIX:
[[75 40 20 10  8  8]
 [47 51 34 11 11 10]
 [18 27 55 27 11 22]
 [ 6 11 11 70 17 29]
 [ 7  5 14 39 65 43]
 [ 7  5 10 27 27 82]]
ACCURACY SCORE:
0.4146
CLASSIFICATION REPORT:
	Precision: 0.4147
	Recall: 0.4169
	F1_Score: 0.4125


In [ ]:
svc_predict = pl_svc.predict(X_pred)
submission_svc= pd.DataFrame()
submission_svc['id']= df_pred.index
submission_svc['difficulty'] = svc_predict
submission_svc.to_csv("submissionsvc.csv", index=False)

###vi. AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier()
pl_abc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', abc)])

pl_abc.fit(X_train, y_train)

y_pred_abc = pl_abc.predict(X_test)

evaluate(y_test, y_pred_abc)

CONFUSION MATRIX:
[[ 36  30  11  82   0   2]
 [ 25  42  23  72   1   1]
 [  7  24  17 106   0   6]
 [  2   9   2 125   2   4]
 [  2   8   7 133   5  18]
 [  3   6   2 120   3  24]]
ACCURACY SCORE:
0.2594
CLASSIFICATION REPORT:
	Precision: 0.3657
	Recall: 0.2725
	F1_Score: 0.2257


In [ ]:
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier()
pl_mlpc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', mlpc)])

pl_mlpc.fit(X_train, y_train)

y_pred_mlpc = pl_mlpc.predict(X_test)

evaluate(y_test, y_pred_mlpc)

CONFUSION MATRIX:
[[93 38 24  5  0  1]
 [64 53 33  6  4  4]
 [35 39 63  8  8  7]
 [19 19 35 46 12 13]
 [21 12 27 28 58 27]
 [20 14 22 23 24 55]]
ACCURACY SCORE:
0.3833
CLASSIFICATION REPORT:
	Precision: 0.4064
	Recall: 0.3829
	F1_Score: 0.3823


In [ ]:
mlpc_predict = pl_mlpc.predict(X_pred)
submission_mlpc= pd.DataFrame()
submission_mlpc['id']= df_pred.index
submission_mlpc['difficulty'] = mlpc_predict
submission_mlpc.to_csv("submissionmlpc.csv", index=False)

#6. Comparing the models

In [1]:
import pandas as pd

df_comparison = pd.DataFrame(columns=['Model_Name', 'Base_Rate', 'Precisions','Recall','F1-Score', 'Accuracy'])


df_comparison['Model_Name'] = ['Logistic Reg', 'KNN', 'Tree', 'Random Forest', 'Support Vector', 'AdaBooster', 'MLPC']
df_comparison['Base_Rate'] = ['0.1694', '0.1694', '0.1694', '0.1694', '0.1694', '0.1694', '0.1694']
df_comparison['Precisions'] = ['0.3944', '0.3089', '0.3325','0.4119', '0.4147', '0.3657', '0.4064']
df_comparison['Recall'] = ['0.3969', '0.2223', '0.3249','0.3715', '0.4169', '0.2725', '0.3829']
df_comparison['F1-Score'] = ['0.3948', '0.1679', '0.3174','0.3664', '0.4125', '0.2257', '0.3823']
df_comparison['Accuracy'] = ['0.3969', '0.2250', '0.3260','0.3719', '0.4146', '0.2594', '0.3833']


df_comparison

,Model_Name,Base_Rate,Precisions,Recall,F1-Score,Accuracy
0,Logistic Reg,0.1694,0.3944,0.3969,0.3948,0.3969
1,KNN,0.1694,0.3089,0.2223,0.1679,0.2250
2,Tree,0.1694,0.3325,0.3249,0.3174,0.3260
3,Random Forest,0.1694,0.4119,0.3715,0.3664,0.3719
4,Support Vector,0.1694,0.4147,0.4169,0.4125,0.4146
5,AdaBooster,0.1694,0.3657,0.2725,0.2257,0.2594
6,MLPC,0.1694,0.4064,0.3829,0.3823,0.3833
